In [1]:
import torch
from models.classifier import TransformerEncoder

In [2]:
hparams = {
    "hidden_size": 48,  # size of the hidden layers and embeddings
    "hidden_ff": 96,  # size of the position-wise feed-forward layer
    "n_encoders": 4,  # number of encoder blocks
    "n_heads": 2,  # number of attention heads in the multiheadattention module
    "n_local": 2,  # number of local attention heads
    "local_window_size": 4,  # size of the window for local attention
    'batch_size': 32,
    "max_length": 256,  # maximum length of the input sequence
    "vocab_size": 11,  # size of the vocabulary
    "learning_rate": 0.001,
    "num_epochs": 30,
    "attention_type": "performer",
    "norm_type": "rezero",
    "num_random_features": 32,  # number of random features for the Attention module (Performer uses this)
    "emb_dropout": 0.1,  # dropout for the embedding block
    "fw_dropout": 0.1,  # dropout for the position-wise feed-forward layer
    "att_dropout": 0.1,  # dropout for the multiheadattention module
    "dc_dropout": 0.1,  # dropout for the decoder block
    "hidden_act": "swish",  # activation function for the hidden layers (attention layers use ReLU)
    "epsilon": 1e-8,
    "weight_decay": 0.01,
    "beta1": 0.9,
    "beta2": 0.999,
}
model = TransformerEncoder(hparams)

# Import fake data

In [3]:
from dataset import DataModule
sequence_path = 'fake_data/sequence_data.parquet'
targets_path = 'fake_data/targets.csv'
test_targets_path = 'fake_data/test_targets.csv'
vocab_path = 'fake_data/vocab.json'

dataloader = DataModule(
    sequence_path=sequence_path, 
    batch_size=hparams['batch_size'],
    target_path=targets_path,
    holdout_target_path=test_targets_path,
    vocab_path=vocab_path,
    subset=False
    )

dataloader.setup()

# Set up training

In [4]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

model_checkpoint = ModelCheckpoint(monitor='val/ap', save_top_k=2, save_last=True, mode='max')
early_stopping = EarlyStopping(monitor='val/ap', patience=100, mode='max')
lr_monitor = LearningRateMonitor(logging_interval='step')
logger = CSVLogger("lightning_logs", name="cls_logs", flush_logs_every_n_steps=1)

trainer = Trainer(max_epochs=10,
                accelerator="cpu",   ### change to "cuda" or "gpu" or 'msp'
                limit_train_batches=0.5,
                logger=logger,
                accumulate_grad_batches=1,
                num_sanity_val_steps=0,
                callbacks = [model_checkpoint, early_stopping, lr_monitor],
                check_val_every_n_epoch=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, dataloader)

Steps per epoch: 100

   | Name        | Type                   | Params | Mode 
----------------------------------------------------------------
0  | transformer | Transformer            | 75.4 K | train
1  | decoder     | CLS_Decoder            | 2.4 K  | train
2  | loss        | BCEWithLogitsLoss      | 0      | train
3  | train_loss  | MeanMetric             | 0      | train
4  | val_loss    | MeanMetric             | 0      | train
5  | test_loss   | MeanMetric             | 0      | train
6  | train_acc   | BinaryAccuracy         | 0      | train
7  | val_acc     | BinaryAccuracy         | 0      | train
8  | test_acc    | BinaryAccuracy         | 0      | train
9  | train_mcc   | BinaryMatthewsCorrCoef | 0      | train
10 | val_mcc     | BinaryMatthewsCorrCoef | 0      | train
11 | test_mcc    | BinaryMatthewsCorrCoef | 0      | train
12 | train_ap    | BinaryAveragePrecision | 0      | train
13 | val_ap      | BinaryAveragePrecision | 0      | train
14 | test_ap     | BinaryAve

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [6]:
trainer.test(model, dataloader.test_dataloader())

/opt/anaconda3/envs/prefer/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test/acc_epoch                 1.0
      test/ap_epoch                 1.0
     test/loss_epoch       0.013805096037685871
     test/mcc_epoch                  1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/loss_epoch': 0.013805096037685871,
  'test/acc_epoch': 1.0,
  'test/mcc_epoch': 1,
  'test/ap_epoch': 1.0}]